## Import libraries and such

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Load in an iris image and test Daugmans integro-differential operator

In [ ]:
img = cv2.imread('test.jpg', 0)